## **Importing Modules**

In [2]:
import keras
from keras.datasets import cifar10
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical, multi_gpu_model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')

In [3]:
keras.backend.set_image_data_format('channels_last')
print(keras.backend.tensorflow_backend._get_available_gpus())

['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


## **Setting Parameters**

In [4]:
nb_classes = 10
nb_epochs = 100
batch_size = 64
img_len = 32
img_width = 32
channels = 3

## **Loading Data**

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

## **Data Preprocessing**

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [7]:
y_test = to_categorical(y_test, num_classes=nb_classes)
y_train = to_categorical(y_train, num_classes=nb_classes)

In [8]:
datagen = ImageDataGenerator(rescale=1.0/255.0, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
train_gen = datagen.flow(x_train, y_train, batch_size=batch_size)
validation_gen = datagen.flow(x_test, y_test, batch_size=batch_size)

In [9]:
train_steps_per_epoch = int(x_train.shape[0]/batch_size)
val_steps_per_epoch = int(x_test.shape[0]/batch_size)

## **Model Architecture**

In [10]:
## Convolutional Model
model = Sequential()

#1st Convolution
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding = "same", kernel_initializer= "he_uniform", input_shape = (img_len, img_width, channels)))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", padding = "same",kernel_initializer= "he_uniform"))
model.add((BatchNormalization()))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#2nd Convolution
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding = "same", kernel_initializer= "he_uniform"))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu", padding = "same", kernel_initializer= "he_uniform"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#3rd Convolution
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding = "same", kernel_initializer= "he_uniform"))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation="relu", padding = "same", kernel_initializer= "he_uniform"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#DNN
model.add(Flatten())
model.add(Dense(units = 128, activation="relu", kernel_initializer="he_uniform"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(nb_classes, activation="softmax"))

In [11]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
__________

In [12]:
## Callbacks
earlystop = EarlyStopping(monitor="val_loss", patience=8)

In [13]:
## Optimizer
opt = SGD(lr = 0.001, momentum=0.9)

In [14]:
## Compile the model
parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])

In [ ]:
## Fitting the model
history = parallel_model.fit_generator(train_gen, validation_data=validation_gen, epochs= nb_epochs, shuffle = True, steps_per_epoch= train_steps_per_epoch, validation_steps=val_steps_per_epoch, callbacks=[earlystop])

Epoch 1/100
781/781 [==============================] - 970s 1s/step - loss: 1.9377 - acc: 0.3378 - val_loss: 1.6330 - val_acc: 0.4323
Epoch 2/100
781/781 [==============================] - 41s 52ms/step - loss: 1.5565 - acc: 0.4421 - val_loss: 1.4012 - val_acc: 0.4963
Epoch 3/100
781/781 [==============================] - 41s 52ms/step - loss: 1.4386 - acc: 0.4819 - val_loss: 1.3643 - val_acc: 0.5063
Epoch 4/100
781/781 [==============================] - 41s 52ms/step - loss: 1.3518 - acc: 0.5149 - val_loss: 1.2833 - val_acc: 0.5381
Epoch 5/100
781/781 [==============================] - 41s 52ms/step - loss: 1.2794 - acc: 0.5416 - val_loss: 1.2548 - val_acc: 0.5539
Epoch 6/100
781/781 [==============================] - 40s 52ms/step - loss: 1.2226 - acc: 0.5631 - val_loss: 1.2212 - val_acc: 0.5601
Epoch 7/100
781/781 [==============================] - 41s 52ms/step - loss: 1.1676 - acc: 0.5824 - val_loss: 1.0805 - val_acc: 0.6145
Epoch 8/100
781/781 [==============================] - 4

781/781 [==============================] - 42s 54ms/step - loss: 0.5761 - acc: 0.8009 - val_loss: 0.5374 - val_acc: 0.8123
Epoch 67/100
781/781 [==============================] - 41s 52ms/step - loss: 0.5720 - acc: 0.8004 - val_loss: 0.5119 - val_acc: 0.8229
Epoch 68/100
781/781 [==============================] - 41s 53ms/step - loss: 0.5714 - acc: 0.8020 - val_loss: 0.5372 - val_acc: 0.8084
Epoch 69/100
781/781 [==============================] - 41s 52ms/step - loss: 0.5708 - acc: 0.8018 - val_loss: 0.5061 - val_acc: 0.8229
Epoch 70/100
781/781 [==============================] - 42s 54ms/step - loss: 0.5635 - acc: 0.8052 - val_loss: 0.5161 - val_acc: 0.8161
Epoch 71/100
781/781 [==============================] - 42s 54ms/step - loss: 0.5613 - acc: 0.8058 - val_loss: 0.5250 - val_acc: 0.8234
Epoch 72/100
781/781 [==============================] - 42s 54ms/step - loss: 0.5592 - acc: 0.8053 - val_loss: 0.5383 - val_acc: 0.8158
Epoch 73/100
781/781 [==============================] - 42s 5

In [19]:
_, acc = parallel_model.evaluate_generator(validation_gen)
print("Validation accuracy is {}".format(round(acc*100, 3)))

Validation accuracy is 84.77
